# Multi-Head Attention Minibenchmark
This benchmark is adapted from [this Pytorch tutorial](https://pytorch.org/tutorials/prototype/nestedtensor.html).

This benchmark expects packages from the `requirements.txt` in the root directory to be installed.

In [8]:
import numpy as np
import timeit
import jax
import flax
from flax import linen as nn
from jax import numpy as jnp
from flax import nnx
import tqdm

In [2]:
class MultiHeadAttention(nn.Module):  #@save
    E_q: int
    E_k: int
    E_v: int    
    E_total: int
    nheads: int
    
    def setup(self):
        rngs = nnx.Rngs(0)

        self.query_proj = nnx.Linear(E_q, E_total, rngs=rngs)
        self.key_proj = nnx.Linear(E_k, E_total, rngs=rngs)
        self.value_proj = nnx.Linear(E_v, E_total, rngs=rngs)
        E_out = self.E_q
        self.out_proj = nnx.Linear(E_total, E_out, rngs=rngs)
        assert E_total % nheads == 0, "Embedding dim is not divisible by nheads"
        self.E_head = E_total // nheads
        self.E_q_last = self.E_q - 1
        self.E_k_last = self.E_k - 1
        self.E_v_last = self.E_v - 1        

    @nn.compact
    def __call__(self, queries, keys, values):
        query = self.query_proj(queries)
        key = self.key_proj(keys)
        value = self.value_proj(values)

        # Step 2. Split heads and prepare for SDPA
        # reshape query, key, value to separate by head
        # (N, L_t, E_total) -> (N, L_t, nheads, E_head) -> (N, nheads, L_t, E_head)
        query = jnp.reshape(query, (query.shape[0], query.shape[1], self.nheads, self.E_head)).transpose(0, 2, 1, 3)
        # (N, L_s, E_total) -> (N, L_s, nheads, E_head) -> (N, nheads, L_s, E_head)
        key = jnp.reshape(key, (key.shape[0], key.shape[1], self.nheads, self.E_head)).transpose(0, 2, 1, 3)
        # (N, L_s, E_total) -> (N, L_s, nheads, E_head) -> (N, nheads, L_s, E_head)
        value = jnp.reshape(value, (value.shape[0], value.shape[1], self.nheads, self.E_head)).transpose(0, 2, 1, 3)

        # Step 3. Run SDPA
        # (N, nheads, L_t, E_head)
        attn_output = jax.nn.dot_product_attention(query, key, value, is_causal=True)
        
        # (N, nheads, L_t, E_head) -> (N, L_t, nheads, E_head) -> (N, L_t, E_total)
        attn_output = jnp.transpose(attn_output, (0, 2, 1, 3))
        attn_output = attn_output.reshape(attn_output.shape[0], attn_output.shape[1], E_total)

        # Step 4. Apply output projection
        # (N, L_t, E_total) -> (N, L_t, E_out)
        attn_output = self.out_proj(attn_output)

        return attn_output

In [3]:
rkey = jax.random.key(1)
N = 512
E_q, E_k, E_v, E_total = 512, 512, 512, 512
E_out = E_q
nheads = 8

In [4]:
def zipf_sentence_lengths(alpha: float, batch_size: int):
    # generate fake corpus by unigram Zipf distribution
    # from wikitext-2 corpus, we get rank "." = 3, "!" = 386, "?" = 858
    sentence_lengths = np.empty(batch_size, dtype=int)
    for ibatch in range(batch_size):
        sentence_lengths[ibatch] = 1
        word = np.random.zipf(alpha)
        while word != 3 and word != 386 and word != 858:
            sentence_lengths[ibatch] += 1
            word = np.random.zipf(alpha)
    return jnp.asarray(sentence_lengths)

In [6]:
def gen_batch(N, E_q, E_k, E_v):
    # generate semi-realistic data using Zipf distribution for sentence lengths
    sentence_lengths = zipf_sentence_lengths(alpha=1.2, batch_size=N)

    # Note: the torch.jagged layout is a nested tensor layout that supports a single ragged
    # dimension and works with torch.compile. The batch items each have shape (B, S*, D)
    # where B = batch size, S* = ragged sequence length, and D = embedding dimension.
    max_l = max(sentence_lengths)
    query = jnp.stack([
        jnp.pad(jax.random.uniform(rkey, (l.item(), E_q)), ((0, max_l - l.item()), (0, 0)))
        for l in sentence_lengths
    ])

    key = jnp.stack([
        jnp.pad(jax.random.uniform(rkey, (l.item(), E_k)), ((0, max_l - l.item()), (0, 0)))
        for l in sentence_lengths
    ])

    value = jnp.stack([
        jnp.pad(jax.random.uniform(rkey, (l.item(), E_v)), ((0, max_l - l.item()), (0, 0)))
        for l in sentence_lengths
    ])

    return query, key, value, sentence_lengths

query, key, value, sentence_lengths = gen_batch(N, E_q, E_k, E_v)

In [7]:
mha = MultiHeadAttention(E_q, E_k, E_v, E_total, nheads)
params = mha.init(rkey, query, key, value)
# mha.apply(params, query, key, value)

## Timing MHA Runtime

In [13]:
def benchmark(func, params, query, key, value):
    begin = timeit.default_timer()
    output = func(params, query, key, value)
    end = timeit.default_timer()
    return output, (end - begin)

num_trials = 100

# warmup
jit_model = jax.jit(mha.apply, backend='gpu').lower(params, query, key, value)
compiled_model = jit_model.compile()
compiled_model(params, query, key, value)

padded_time = []

for _ in tqdm.trange(1, num_trials + 1):
    _, t = benchmark(compiled_model, params, query, key, value)
    padded_time.append(t)
print("Average padded tensor runtime:", sum(padded_time) / len(padded_time), "seconds")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 550.41it/s]

Average padded tensor runtime: 0.0018002113793045282 seconds


## Timing MHA JIT

In [17]:
def benchmark_jit(func, params, query, key, value):
    jax.clear_caches()
    begin = timeit.default_timer()
    jit_model = jax.jit(func.apply, backend='gpu').lower(params, query, key, value)
    compiled_model = jit_model.compile()
    end = timeit.default_timer()
    return output, (end - begin)

num_trials = 10

# warmup
for _ in range(5):
    benchmark_jit(mha, params, query, key, value)
    
jit_time = []

for _ in tqdm.trange(1, num_trials + 1):
    _, t = benchmark_jit(mha, params, query, key, value)
    jit_time.append(t)
print("Average padded tensor JIT time:", sum(jit_time) / len(jit_time), "seconds")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.14it/s]

Average padded tensor JIT time: 0.3156730689108372 seconds
